In [7]:
import os
import shutil
from pathlib import Path

def decimal_to_custom_base(n):
    """将十进制数字转换为25进制，使用非容易混淆的字符编码"""
    characters = 'acdefhjkmnpqrtuvwxyz23478'
    base = len(characters)
    if n == 0:
        return characters[0]  # 将0映射到第一个字符'a'
    result = ""
    while n > 0:
        n, remainder = divmod(n, base)
        result = characters[remainder] + result
    return result

def move_and_rename_images(directory):
    """
    递归遍历指定目录，将所有图片文件移动到该目录并重命名。
    完成后，根据成功处理的文件记录，删除非图片文件和未成功处理的图片及空目录。
    同时记录操作到操作日志文件。
    
    图片文件被重命名为基于26进制的字母组合，如'a', 'b', ..., 'z', 'aa', 'ab', ...
    
    参数:
    directory (str or Path): 目标目录路径
    """
    counter = 0
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
    successful_files = set()
    operation_log_path = Path(directory) / "operation_log.txt"

    target_dir = Path(directory)
    if not target_dir.is_dir():
        print(f"目录不存在: {directory}")
        return

    failed_counter = 0  # 记录失败操作的文件数量

    with open(operation_log_path, 'w') as log_file:
        for root, dirs, files in os.walk(directory, topdown=False):
            for name in files:
                if any(name.lower().endswith(ext) for ext in image_extensions):
                    counter += 1
                    source_path = Path(root) / name
                    extension = source_path.suffix
                    new_name = decimal_to_custom_base(counter)
                    target_path = target_dir / f"{new_name}{extension}"
                    
                    try:
                        shutil.move(str(source_path), str(target_path))
                        print(f"移动并重命名: {source_path} -> {target_path}")
                        successful_files.add(new_name + extension)  # 更新为包含扩展名
                        log_file.write(f"{source_path},{target_path}\n")
                    except Exception as e:
                        print(f"处理文件{source_path}时发生错误: {e}")
                        failed_counter += 1  # 失败操作的文件数量加一

    def clean_non_image_or_unprocessed_files_and_dirs(path, successful_files_set):
        for item in path.iterdir():
            if item.is_file():
                if item.suffix.lower() not in image_extensions or item.name not in successful_files_set:
                    try:
                        if item != operation_log_path:
                            item.unlink()
                            print(f"删除文件: {item}")
                    except Exception as e:
                        print(f"删除文件{item}时发生错误: {e}")
            elif item.is_dir():
                clean_non_image_or_unprocessed_files_and_dirs(item, successful_files_set)
                try:
                    if not any(item.iterdir()) and item != target_dir:
                        item.rmdir()
                        print(f"删除空目录: {item}")
                except Exception as e:
                    print(f"删除空目录{item}时发生错误: {e}")
                    
    clean_non_image_or_unprocessed_files_and_dirs(target_dir, successful_files)

    # 输出成功操作的文件数量和失败操作的文件数量
    total_files = counter
    successful_files_count = len(successful_files)
    failed_files_count = failed_counter
    print(f"处理完成。成功操作文件数量: {successful_files_count}，失败操作文件数量: {failed_files_count}，总共操作文件数量: {total_files}")

def reverse_operations(directory):
    """
    逆向操作，根据操作日志文件中的记录还原文件至原始位置。
    修正：正确处理源路径和目标路径，确保文件从新位置移回到原始位置。
    """
    operation_log_path = Path(directory) / "operation_log.txt"
    if not operation_log_path.exists():
        print("操作日志文件不存在，无法进行逆向操作。")
        return
    
    with open(operation_log_path, 'r') as log_file:
        for line in log_file:
            source_path_str, target_path_str = line.strip().split(',')
            source_path = Path(source_path_str)  # 原始路径
            target_path = Path(target_path_str)     # 当前路径（正向操作后的位置）
            
            try:
                # 确保源路径的父目录存在，因为我们要将文件移回那里
                source_path.parent.mkdir(parents=True, exist_ok=True)
                shutil.move(str(target_path), str(source_path))  # 从当前路径移回原始路径
                print(f"逆向操作完成: {target_path} -> {source_path}")
            except FileNotFoundError:
                print(f"文件{target_path}不存在，跳过逆向操作。")
            except Exception as e:
                print(f"逆向操作时发生错误: {e}")

In [8]:
target_directory = "/Users/chenweichu/dev/data/test19"
print("开始处理...")
move_and_rename_images(target_directory)
# 可以手动调用逆向操作，但需谨慎处理
# reverse_operations(target_directory)

开始处理...
移动并重命名: /Users/chenweichu/dev/data/test19/45/_storage_emulated_0_sina_weibo_storage_photoalbum_save_weibo_img-68411e4f860b075bc294fbd04306823a.jpg -> /Users/chenweichu/dev/data/test19/c.jpg
移动并重命名: /Users/chenweichu/dev/data/test19/45/_storage_emulated_0_sina_weibo_storage_photoalbum_save_weibo_img-a0d23d3cc2497bf45a3882a168a52c9f.jpg -> /Users/chenweichu/dev/data/test19/d.jpg
移动并重命名: /Users/chenweichu/dev/data/test19/45/_storage_emulated_0_sina_weibo_storage_photoalbum_save_weibo_img-2673fb9d47f847c92853dd9173b8840f.jpg -> /Users/chenweichu/dev/data/test19/e.jpg
移动并重命名: /Users/chenweichu/dev/data/test19/45/1.jpg -> /Users/chenweichu/dev/data/test19/f.jpg
移动并重命名: /Users/chenweichu/dev/data/test19/45/0.jpg -> /Users/chenweichu/dev/data/test19/h.jpg
移动并重命名: /Users/chenweichu/dev/data/test19/未命名文件夹/45/kristy204.jpg -> /Users/chenweichu/dev/data/test19/j.jpg
移动并重命名: /Users/chenweichu/dev/data/test19/未命名文件夹/45/kristy207.jpg -> /Users/chenweichu/dev/data/test19/k.jpg
移动并重命名: /Users/c